<a href="https://colab.research.google.com/github/2403a52267/NLP_2267/blob/main/2267_B10_NLP_A6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
d=pd.read_excel('/content/LDA-Data.xlsx')
print(d)

                             News
0   Virat scored century in match
1            BJP won in elections
2  Bumra took 5 wicket in a match
3  Congress form state government


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stopwords_set = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers (keeping only alphabets)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize the text
    words = nltk.word_tokenize(text)
    # Remove stopwords and perform lemmatization
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords_set]
    # Rejoin words into a string
    return ' '.join(processed_words)

# Apply the preprocessing function to the 'News' column
d['Processed_News'] = d['News'].apply(preprocess_text)

# Display the original and processed news
print("Original News:")
print(d['News'])
print("\nProcessed News:")
print(d['Processed_News'])

Original News:
0     Virat scored century in match
1              BJP won in elections
2    Bumra took 5 wicket in a match
3    Congress form state government
Name: News, dtype: object

Processed News:
0        virat scored century match
1                      bjp election
2           bumra took wicket match
3    congress form state government
Name: Processed_News, dtype: object


**LDA with Sample Data and BoW**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Re-create CountVectorizer and bow_matrix to ensure availability
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Number of topics to extract
num_topics = 2

# Initialize LDA model
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# Fit LDA model to the Bag of Words matrix
lda_model.fit(bow_matrix)

# Display the topics and their top keywords
print("Topics and their top keywords:")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_): # Corrected from components__ to components_
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lda_model, vectorizer.get_feature_names_out(), 5)

Topics and their top keywords:
Topic 1:
form government congress state election
Topic 2:
match virat century scored took


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming 'd' DataFrame with 'Processed_News' column is already available

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Processed_News' column
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("Bag of Words Matrix (DataFrame):")
display(bow_df)

Bag of Words Matrix (DataFrame):


,bjp,bumra,century,congress,election,form,government,match,scored,state,took,virat,wicket
0,0,0,1,0,0,0,0,1,1,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,1,0,1,1,0,0,1,0,0,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Processed_News' column
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("Bag of Words Matrix:")
display(bow_df)

print("\nVocabulary (Feature Names):")
print(vectorizer.get_feature_names_out())

Bag of Words Matrix:


,bjp,bumra,century,congress,election,form,government,match,scored,state,took,virat,wicket
0,0,0,1,0,0,0,0,1,1,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,1,0,1,1,0,0,1,0,0,0



Vocabulary (Feature Names):
['bjp' 'bumra' 'century' 'congress' 'election' 'form' 'government' 'match'
 'scored' 'state' 'took' 'virat' 'wicket']


In [ ]:
import pandas as pd
d=pd.read_csv('/content/arxiv_data.csv')
print(d)

                                                  titles  \
0      Survey on Semantic Stereo Matching / Semantic ...   
1      FUTURE-AI: Guiding Principles and Consensus Re...   
2      Enforcing Mutual Consistency of Hard Regions f...   
3      Parameter Decoupling Strategy for Semi-supervi...   
4      Background-Foreground Segmentation for Interio...   
...                                                  ...   
51769  Hierarchically-coupled hidden Markov models fo...   
51770                         Blinking Molecule Tracking   
51771  Towards a Mathematical Foundation of Immunolog...   
51772  A Semi-Automatic Graph-Based Approach for Dete...   
51773  SparseCodePicking: feature extraction in mass ...   

                                               summaries  \
0      Stereo matching is one of the widely used tech...   
1      The recent advancements in artificial intellig...   
2      In this paper, we proposed a novel mutual cons...   
3      Consistency training has proven 

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stopwords_set = set(stopwords.words('english'))

def preprocess_text(text):
    # Ensure text is a string to prevent errors with NaN or other types
    if pd.isna(text):
        return ""
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers (keeping only alphabets)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize the text
    words = nltk.word_tokenize(text)
    # Remove stopwords and perform lemmatization
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords_set]
    # Rejoin words into a string
    return ' '.join(processed_words)

# Apply the preprocessing function to the 'titles' column (assuming 'titles' is the intended column)
d['Processed_News'] = d['titles'].apply(preprocess_text)

# Display the original and processed news
print("Original Titles:")
print(d['titles'].head())
print("\nProcessed Titles (now in 'Processed_News' column):")
print(d['Processed_News'].head())

Original Titles:
0    Survey on Semantic Stereo Matching / Semantic ...
1    FUTURE-AI: Guiding Principles and Consensus Re...
2    Enforcing Mutual Consistency of Hard Regions f...
3    Parameter Decoupling Strategy for Semi-supervi...
4    Background-Foreground Segmentation for Interio...
Name: titles, dtype: object

Processed Titles (now in 'Processed_News' column):
0    survey semantic stereo matching semantic depth...
1    future ai guiding principle consensus recommen...
2    enforcing mutual consistency hard region semi ...
3    parameter decoupling strategy semi supervised ...
4    background foreground segmentation interior se...
Name: Processed_News, dtype: object


**LDA with Kaggle Data and BoW**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Re-create CountVectorizer and bow_matrix to ensure availability
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Number of topics to extract
num_topics = 2

# Initialize LDA model
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# Fit LDA model to the Bag of Words matrix
lda_model.fit(bow_matrix)

# Display the topics and their top keywords
print("Topics and their top keywords:")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_): # Corrected from components__ to components_
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lda_model, vectorizer.get_feature_names_out(), 5)

Topics and their top keywords:
Topic 1:
learning network graph deep reinforcement
Topic 2:
image network detection object segmentation


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming 'd' DataFrame with 'Processed_News' column is already available

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Processed_News' column
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("Bag of Words Matrix (DataFrame):")
display(bow_df)

Bag of Words Matrix (DataFrame):


,aa,aaa,aabo,aacp,aad,aads,aaformer,aag,aam,aamdrl,...,zoo,zoom,zooming,zoomnet,zoonosis,zooplankton,zope,zorro,zpd,zstgan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51772,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Processed_News' column
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("Bag of Words Matrix:")
display(bow_df)

print("\nVocabulary (Feature Names):")
print(vectorizer.get_feature_names_out())

Bag of Words Matrix:


,aa,aaa,aabo,aacp,aad,aads,aaformer,aag,aam,aamdrl,...,zoo,zoom,zooming,zoomnet,zoonosis,zooplankton,zope,zorro,zpd,zstgan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51772,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



Vocabulary (Feature Names):
['aa' 'aaa' 'aabo' ... 'zorro' 'zpd' 'zstgan']


**NMF with Kaggle Data and BoW**

In [ ]:
from sklearn.decomposition import NMF

# Initialize NMF model
nmf_model = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the Bag of Words matrix
nmf_model.fit(bow_matrix)

print("NMF model fitted successfully.")

NMF model fitted successfully.


In [ ]:
print("Topics and their top keywords for NMF:")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(nmf_model, vectorizer.get_feature_names_out(), 5)

Topics and their top keywords for NMF:
Topic 1:
learning reinforcement deep representation based
Topic 2:
network neural image graph using


**NMF with Kaggle Data and BoW**

In [ ]:
d_lda = pd.read_excel('/content/LDA-Data.xlsx')
print(d_lda.head())

                             News
0   Virat scored century in match
1            BJP won in elections
2  Bumra took 5 wicket in a match
3  Congress form state government


In [ ]:
d_lda['Processed_News'] = d_lda['News'].apply(preprocess_text)

print("Original News (from d_lda):")
print(d_lda['News'].head())
print("\nProcessed News (from d_lda):")
print(d_lda['Processed_News'].head())

Original News (from d_lda):
0     Virat scored century in match
1              BJP won in elections
2    Bumra took 5 wicket in a match
3    Congress form state government
Name: News, dtype: object

Processed News (from d_lda):
0        virat scored century match
1                      bjp election
2           bumra took wicket match
3    congress form state government
Name: Processed_News, dtype: object


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer_lda = CountVectorizer() # Use a new vectorizer instance to avoid conflicts

# Fit and transform the 'Processed_News' column from d_lda
bow_matrix_lda = vectorizer_lda.fit_transform(d_lda['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df_lda = pd.DataFrame(bow_matrix_lda.toarray(), columns=vectorizer_lda.get_feature_names_out())

print("Bag of Words Matrix for d_lda:")
display(bow_df_lda)

print("\nVocabulary (Feature Names) for d_lda:")
print(vectorizer_lda.get_feature_names_out())

Bag of Words Matrix for d_lda:


,bjp,bumra,century,congress,election,form,government,match,scored,state,took,virat,wicket
0,0,0,1,0,0,0,0,1,1,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,1,0,1,1,0,0,1,0,0,0



Vocabulary (Feature Names) for d_lda:
['bjp' 'bumra' 'century' 'congress' 'election' 'form' 'government' 'match'
 'scored' 'state' 'took' 'virat' 'wicket']


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Number of topics to extract (assuming num_topics is already defined from a previous step)
# num_topics = 2 # If num_topics was not already defined, it would need to be here.

# Initialize LDA model
lda_model_lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# Fit LDA model to the Bag of Words matrix for d_lda
lda_model_lda.fit(bow_matrix_lda)

print("LDA model fitted successfully to d_lda data.")

# Display the topics and their top keywords
print("Topics and their top keywords for d_lda:")

def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lda_model_lda, vectorizer_lda.get_feature_names_out(), 5)

LDA model fitted successfully to d_lda data.
Topics and their top keywords for d_lda:
Topic 1:
form government congress state election
Topic 2:
match virat century scored took


In [ ]:
from sklearn.decomposition import NMF

# Initialize NMF model
nmf_model_lda = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the Bag of Words matrix for d_lda
nmf_model_lda.fit(bow_matrix_lda)

print("NMF model fitted successfully to d_lda data.")

# Display the topics and their top keywords for d_lda using NMF
print("Topics and their top keywords for d_lda (NMF):")
display_topics(nmf_model_lda, vectorizer_lda.get_feature_names_out(), 5)

NMF model fitted successfully to d_lda data.
Topics and their top keywords for d_lda (NMF):
Topic 1:
match virat took scored wicket
Topic 2:
state form congress government election


**NMF with Sample Data and TFIDF**

In [ ]:
print(d_lda.head())

                             News                  Processed_News
0   Virat scored century in match      virat scored century match
1            BJP won in elections                    bjp election
2  Bumra took 5 wicket in a match         bumra took wicket match
3  Congress form state government  congress form state government


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Processed_News' column to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(d_lda['Processed_News'])

# Convert the TF-IDF matrix to a DataFrame for better readability (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print("TF-IDF Matrix (first 5 rows and columns):")
print(tfidf_df.head())

print("\nVocabulary (Feature Names) from TF-IDF:")
print(tfidf_vectorizer.get_feature_names_out())

TF-IDF Matrix (first 5 rows and columns):
        bjp     bumra   century  congress  election  form  government  \
0  0.000000  0.000000  0.525473       0.0  0.000000   0.0         0.0   
1  0.707107  0.000000  0.000000       0.0  0.707107   0.0         0.0   
2  0.000000  0.525473  0.000000       0.0  0.000000   0.0         0.0   
3  0.000000  0.000000  0.000000       0.5  0.000000   0.5         0.5   

      match    scored  state      took     virat    wicket  
0  0.414289  0.525473    0.0  0.000000  0.525473  0.000000  
1  0.000000  0.000000    0.0  0.000000  0.000000  0.000000  
2  0.414289  0.000000    0.0  0.525473  0.000000  0.525473  
3  0.000000  0.000000    0.5  0.000000  0.000000  0.000000  

Vocabulary (Feature Names) from TF-IDF:
['bjp' 'bumra' 'century' 'congress' 'election' 'form' 'government' 'match'
 'scored' 'state' 'took' 'virat' 'wicket']


In [ ]:
from sklearn.decomposition import NMF

# num_topics is already defined from a previous step as 2

# Initialize NMF model
nmf_model_tfidf = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the TF-IDF matrix
nmf_model_tfidf.fit(tfidf_matrix)

print("NMF model fitted successfully to TF-IDF data.")

NMF model fitted successfully to TF-IDF data.


In [ ]:
print("Topics and their top keywords for d_lda (NMF on TF-IDF):")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(nmf_model_tfidf, tfidf_vectorizer.get_feature_names_out(), 5)

Topics and their top keywords for d_lda (NMF on TF-IDF):
Topic 1:
match bumra wicket took virat
Topic 2:
election bjp form government state


**NMF with Kaggle Data and TFIDF**

In [ ]:
print(d.head())

                                              titles  \
0  Survey on Semantic Stereo Matching / Semantic ...   
1  FUTURE-AI: Guiding Principles and Consensus Re...   
2  Enforcing Mutual Consistency of Hard Regions f...   
3  Parameter Decoupling Strategy for Semi-supervi...   
4  Background-Foreground Segmentation for Interio...   

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                         terms  \
0           ['cs.CV', 'cs.LG']   
1  ['cs.CV', 'cs.AI', 'cs.LG']   
2           ['cs.CV', 'cs.AI']   
3                    ['cs.CV']   
4           ['cs.CV', 'cs.LG']   

                                      Processed_News  
0  survey semantic stereo matching semantic depth...  
1  future a

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer for the 'd' DataFrame
tfidf_vectorizer_arxiv = TfidfVectorizer()

# Fit and transform the 'Processed_News' column of 'd' to create the TF-IDF matrix
tfidf_matrix_arxiv = tfidf_vectorizer_arxiv.fit_transform(d['Processed_News'])

print("TF-IDF matrix for arxiv data created successfully.")
print("Shape of TF-IDF matrix:", tfidf_matrix_arxiv.shape)
print("Number of features (unique words):")
print(len(tfidf_vectorizer_arxiv.get_feature_names_out()))

TF-IDF matrix for arxiv data created successfully.
Shape of TF-IDF matrix: (51774, 16440)
Number of features (unique words):
16440


In [ ]:
from sklearn.decomposition import NMF

# num_topics is already defined from a previous step as 2

# Initialize NMF model for the arxiv data
nmf_model_tfidf_arxiv = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the TF-IDF matrix for arxiv data
nmf_model_tfidf_arxiv.fit(tfidf_matrix_arxiv)

print("NMF model fitted successfully to arxiv TF-IDF data.")

NMF model fitted successfully to arxiv TF-IDF data.


In [ ]:
print("Topics and their top keywords for arxiv data (NMF on TF-IDF):")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(nmf_model_tfidf_arxiv, tfidf_vectorizer_arxiv.get_feature_names_out(), 10)

Topics and their top keywords for arxiv data (NMF on TF-IDF):
Topic 1:
network neural graph image convolutional adversarial detection generative object attention
Topic 2:
learning reinforcement deep representation model based supervised unsupervised multi transfer
